In [7]:
from loren_frank_data_processing import (make_epochs_dataframe,
                                         make_neuron_dataframe,
                                         make_tetrode_dataframe)
from src.parameters import (ANIMALS, MIN_N_NEURONS, _BRAIN_AREAS)

from tqdm.auto import tqdm
import numpy as np  

In [2]:
epoch_info = make_epochs_dataframe(ANIMALS)
neuron_info = make_neuron_dataframe(ANIMALS)

neuron_info = neuron_info.loc[
(neuron_info.type == 'principal') &
(neuron_info.numspikes > 100) &
neuron_info.area.isin(_BRAIN_AREAS)]

n_neurons = (neuron_info
                .groupby(['animal', 'day', 'epoch'])
                .neuron_id
                .agg(len)
                .rename('n_neurons')
                .to_frame())

epoch_info = epoch_info.join(n_neurons)
is_w_track = (epoch_info.environment
                .isin(['TrackA', 'TrackB', 'WTrackA', 'WTrackB']))
is_animal = epoch_info.index.isin(['bon', 'fra', 'gov', 'dud', 'con', 'Cor', 'dav', 'egy', 'cha'], level='animal')

valid_epochs = (is_w_track &
                (epoch_info.n_neurons > MIN_N_NEURONS) &
                is_animal
                )

In [ ]:
num_tetrodes_per_epoch = []

for epoch_key in tqdm(epoch_info[valid_epochs].index, desc='epochs'):
    animal, day, epoch = epoch_key
    
    #print(f'Getting tetrode info for Animal: {animal}, Day: {day}, Epoch: {epoch}')
    
    tetrode_info = make_tetrode_dataframe(ANIMALS, epoch_key=epoch_key)
    is_brain_areas = (
        tetrode_info.area.astype(str).str.upper().isin(_BRAIN_AREAS))
    tetrode_info = tetrode_info.loc[is_brain_areas]

    #save number of tetrodes into a list
    num_tetrodes_per_epoch.append(len(tetrode_info))

In [48]:
#get number of valid epochs in total
num_valid_epochs = len(epoch_info[valid_epochs])
print(f'Number of valid epochs: {num_valid_epochs}')

#get the number of valid epochs per day
num_valid_epochs_per_day = epoch_info[valid_epochs].groupby(['animal', 'day']).size()
#get the mean and std of valid epochs per day
mean_valid_epochs_per_day = np.mean(num_valid_epochs_per_day)
std_valid_epochs_per_day = np.std(num_valid_epochs_per_day)
print(f'Mean number of valid epochs per day: {mean_valid_epochs_per_day}')
print(f'Std of number of valid epochs per day: {std_valid_epochs_per_day}')

#for each animal, count the number of recording days from num_valid_epochs_per_day
num_recording_days = num_valid_epochs_per_day.groupby('animal').size()
#get the mean and std of recording days
mean_recording_days = np.mean(num_recording_days)
std_recording_days = np.std(num_recording_days)
print(f'Mean number of recording days: {mean_recording_days}')
print(f'Std of number of recording days: {std_recording_days}')

Number of valid epochs: 140
Mean number of valid epochs per day: 2.641509433962264
Std of number of valid epochs per day: 0.7292886071731203
Mean number of recording days: 5.888888888888889
Std of number of recording days: 2.3778816176702997
